# RSA deterministic outputs: Brute-forcing RSA

Create a Python program that uses the RSA implementation you created in exercise 1 to brute force an RSA-encrypted word. The program should take a public key, the RSA encrypted ciphertext and the length of the word as the inputs.

By running some experiments, measure the average time it takes to brute force an all lowercase four-letter word vs. an all lowercase five-letter word. How many times longer does it take and why? 

For how long would it take to try all possible all lowercase six-letter words? 

Modify your brute-force program by adding an option to pass a dictionary as input for trying arbitrary English words.

In [ ]:
import argparse
import string
import sys
from typing import Iterable
from itertools import product
import gmpy2
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicKey
from cryptography.hazmat.primitives.serialization import load_pem_public_key



In [ ]:
def bytes_to_int(text: bytes):
    return int.from_bytes(text, 'big')

def int_to_bytes(num):
    num_int = int(num)
    return int.to_bytes(num_int, (num_int.bit_length()+7)//8, "big")


def encrypt(plain: bytes, e: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(plain[i:i+block_size])
              for i in range(0, len(plain), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, e, n)) for block in blocks])

def bruteforce(key: RSAPublicKey, cipher: bytes, search_space: Iterable):
    public_numbers = key.public_numbers()
    for word_tuple in search_space:
        word = "".join(word_tuple)
        attempt = encrypt(word.encode(),
                          public_numbers.e, public_numbers.n)

        if attempt == cipher:
            return word

    return None



In [ ]:
def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-c", "--ciphertext", help="Encrypted word", type=str, required=True)
    parser.add_argument(
        "-l", "--length", help="Length of the word", type=int)
    parser.add_argument(
        "-pub", "--public-key", help="File containing the public key", type=argparse.FileType('r'), required=True)
    parser.add_argument(
        "-d", "--dictionary", help="File containing the dictionary for the attack", type=argparse.FileType('r'))

    args = parser.parse_args(arguments)

    public_key = load_pem_public_key(args.public_key.read().encode())

    if args.dictionary:
        search_space = args.dictionary.read().splitlines()
        if args.length:
            search_space = [
                word for word in search_space if len(word) == args.length]
    else:
        if args.length is not None:
            search_space = product(string.ascii_lowercase, repeat=args.length)
        else:
            parser.error(
                "-l/--length is required  when -d/--dictionary is not provided")

    plain = bruteforce(public_key, bytes.fromhex(
        args.ciphertext), search_space)
    if plain:
        print(plain)
    else:
        print("Word not found")



In [ ]:

sys.argv = [
    "", 
    "-c", "afb7c2810bc4687588ff93559b4b89d17c6c1b33e421c06d11c8675da4412adaf845de8d94366630f9804df799c1ff5bdc9619261b7448e1638461cfa4d459e21cbb898c1ffbfe599622b9bc5a97496759f6e8e20ed3380600239ecdb1537c48ad735630e816512faecc5e0f126e8ea583b06fa2dd989569c10783674fe0bb486731477247cef855696df351cea93d60718f610bb0473c39c2c9baf06751b6bcf8675ad151d37807aedddd1fae65990729a47da505b284a0f329d1dd4ae8db9b4d9a5c6204ae1c6b1a2c3987645242ba1c4b52a9cf27e8c18e7f00275c9f0577af959869464ebfaa4d0b029059cb94eca3f8d559598a83d5207f5483cae8bfb4",
    "-pub", "InputFiles/RSA_powmod_without_padding/public_key.pem",
    "-d" , "dictionary_1000.txt"
    

]
main(sys.argv[1:])

    #"-d" , "dictionary_1000.txt"
    #"-l" , 5

In [ ]:
with open("InputFiles/RSA_powmod_without_padding/public_key.pem", "r") as pu_key_file:
    print(pu_key_file.read())